In [3]:
import time
import torch
from torch import nn, optim
import d2lzh_pytorch as d2l
import torch.nn.functional as F

import sys
sys.path.append("..")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

AttributeError: module 'torch' has no attribute '_utils_internal'

In [21]:
def nin_block(in_channels, out_channels, kernel_size, stride, padding):
    blk = nn.Sequential(nn.Conv2d(in_channels,out_channels, kernel_size, stride, padding),
                        nn.ReLU(),
                        nn.Conv2d(out_channels,out_channels, kernel_size=1),
                        nn.ReLU(),
                        nn.Conv2d(out_channels,out_channels,kernel_size=1),
                        nn.ReLU()
                       )
    return blk

In [24]:
class GlobalAvgPool2d(nn.Module):
    def __init__(self):
        super(GlobalAvgPool2d,self).__init__()
        
    def forward(delf, X):
        return F.avg_pool2d(X, kernel_size=X.size()[2:])

net = nn.Sequential(
    nin_block(1, 96, kernel_size=11,stride=4, padding=0),
    nn.MaxPool2d(kernel_size=3,stride=2),
    nin_block(96, 256, kernel_size=5, stride=1,padding=2),
    nn.MaxPool2d(kernel_size=3,stride=2),
    nin_block(256, 384, kernel_size=3, stride=1, padding=1),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nn.Dropout(0.5),
    nin_block(384,10,kernel_size=3,stride=1,padding=1),
    GlobalAvgPool2d(),
    d2l.FlattenLayer()
)

In [25]:
X = torch.rand(1, 1, 224, 224)
for name, blk in net.named_children():
    X = blk(X)
    print(name, 'output shape: ', X.shape)

0 output shape:  torch.Size([1, 96, 54, 54])
1 output shape:  torch.Size([1, 96, 26, 26])
2 output shape:  torch.Size([1, 256, 26, 26])
3 output shape:  torch.Size([1, 256, 12, 12])
4 output shape:  torch.Size([1, 384, 12, 12])
5 output shape:  torch.Size([1, 384, 5, 5])
6 output shape:  torch.Size([1, 384, 5, 5])
7 output shape:  torch.Size([1, 10, 5, 5])
8 output shape:  torch.Size([1, 10, 1, 1])
9 output shape:  torch.Size([1, 10])


In [26]:
batch_size = 128
# 如出现“out of memory”的报错信息，可减⼩batch_size或resize
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size,
resize=224)
lr, num_epochs = 0.002, 5
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
d2l.train_ch5(net, train_iter, test_iter, batch_size, optimizer,device, num_epochs)




training on  cpu


KeyboardInterrupt: 

In [37]:
X = torch.rand(1,2,3,4,5,6)

In [39]:
print(X.size()[2:])

torch.Size([3, 4, 5, 6])
